In [1]:
# Libraries
import pandas as pd

In [2]:
# Ingest data
raw = pd.read_csv("ce.data.0.AllCESSeries",
                 sep="\t",          # file is tab separated
                 dtype=str,         # keep all columns as strings (preserve codes that lead with zeroes)
                 na_values=["."])   # treat . as a missing value

series = pd.read_csv("ce.series", sep="\t", dtype=str)
industry = pd.read_csv("ce.industry", sep="\t", dtype=str)
data_type = pd.read_csv("ce.datatype.txt", sep="\t", dtype=str)
month = pd.read_csv("ce.period", sep="\t", dtype=str)
sector = pd.read_csv("ce.supersector.txt", sep="\t", dtype=str) 
                      

# Remove extra space in column names
raw.columns = raw.columns.str.strip()
series.columns = series.columns.str.strip()
industry.columns = industry.columns.str.strip()
data_type.columns = data_type.columns.str.strip()
month.columns = month.columns.str.strip()
sector.columns = sector.columns.str.strip()

In [3]:
# Merge
df = (
    raw
    .merge(series, on="series_id", how="left")          # adds: series_title, seasonal, supersector_code, industry_code, data_type_code, footnote_codes, begin_year, begin_period, end_year, end_period, naics_code, publishing_status
    .merge(industry, on="industry_code", how="left")    # adds: industry_name, display_level, selectable, sort_sequence
    .merge(data_type, on="data_type_code", how="left")  # adds: data_type_text 
    .merge(month, on="period", how="left")              # adds: period_name
    .merge(sector, on="supersector_code", how="left")   # adds: supersector_name
)

df.head()

,series_id,year,period,value,footnote_codes_x,supersector_code,industry_code,data_type_code,seasonal,series_title,...,naics_code,publishing_status,industry_name,display_level,selectable,sort_sequence,data_type_text,mm,month,supersector_name
0,CES0000000001,1939,M01,29923,NaN,00,00000000,01,S,"All employees, thousands, total nonfarm, seaso...",...,-,B,Total nonfarm,0,T,1,"ALL EMPLOYEES, THOUSANDS",JAN,January,Total nonfarm
1,CES0000000001,1939,M02,30100,NaN,00,00000000,01,S,"All employees, thousands, total nonfarm, seaso...",...,-,B,Total nonfarm,0,T,1,"ALL EMPLOYEES, THOUSANDS",FEB,February,Total nonfarm
2,CES0000000001,1939,M03,30280,NaN,00,00000000,01,S,"All employees, thousands, total nonfarm, seaso...",...,-,B,Total nonfarm,0,T,1,"ALL EMPLOYEES, THOUSANDS",MAR,March,Total nonfarm
3,CES0000000001,1939,M04,30094,NaN,00,00000000,01,S,"All employees, thousands, total nonfarm, seaso...",...,-,B,Total nonfarm,0,T,1,"ALL EMPLOYEES, THOUSANDS",APR,April,Total nonfarm
4,CES0000000001,1939,M05,30299,NaN,00,00000000,01,S,"All employees, thousands, total nonfarm, seaso...",...,-,B,Total nonfarm,0,T,1,"ALL EMPLOYEES, THOUSANDS",MAY,May,Total nonfarm


In [4]:
# Keep important columns
keep_cols = [
    "series_id",
    "industry_name",
    "supersector_name",
    "naics_code",
    "year",
    "month",
    "value",
    "seasonal",
    "month_num",
]

df = df.copy()

# Keep only employment levels (data_type_code = 1)
df = df[df["data_type_code"] == "01"].copy()

# Fix types
df["value"] = pd.to_numeric(df["value"], errors="coerce")
df["year"]  = pd.to_numeric(df["year"],  errors="coerce")

# Map period codes to month numbers and names
period_to_num = {
    "M01": "01", "M02": "02", "M03": "03", "M04": "04",
    "M05": "05", "M06": "06", "M07": "07", "M08": "08",
    "M09": "09", "M10": "10", "M11": "11", "M12": "12"
}
period_to_name = {
    "M01": "January", "M02": "February", "M03": "March", "M04": "April",
    "M05": "May", "M06": "June", "M07": "July", "M08": "August",
    "M09": "September", "M10": "October", "M11": "November", "M12": "December"
}

# Remove annual M13 values
df = df[df["period"].isin(period_to_num.keys())].copy()

# Add month number + full month name
df["month_num"] = df["period"].map(period_to_num)
df["month"]     = df["period"].map(period_to_name)

# Filter to relevant years
df = df[df["year"] >= 1995].copy()

# Final filtered set
df_filtered = df[keep_cols].copy()

df_filtered.head()

,series_id,industry_name,supersector_name,naics_code,year,month,value,seasonal,month_num
672,CES0000000001,Total nonfarm,Total nonfarm,-,1995,January,116505.0,S,01
673,CES0000000001,Total nonfarm,Total nonfarm,-,1995,February,116694.0,S,02
674,CES0000000001,Total nonfarm,Total nonfarm,-,1995,March,116908.0,S,03
675,CES0000000001,Total nonfarm,Total nonfarm,-,1995,April,117065.0,S,04
676,CES0000000001,Total nonfarm,Total nonfarm,-,1995,May,117047.0,S,05


In [5]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 620450 entries, 672 to 8083955
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   series_id         620450 non-null  object 
 1   industry_name     620450 non-null  object 
 2   supersector_name  620450 non-null  object 
 3   naics_code        620450 non-null  object 
 4   year              620450 non-null  int64  
 5   month             620450 non-null  object 
 6   value             620450 non-null  float64
 7   seasonal          620450 non-null  object 
 8   month_num         620450 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 47.3+ MB


In [6]:
# Save unfiltered data
df_filtered.to_csv("bls_employment_stats.csv", index=False) 